# PROMOTED PRODUCTS - MBR

In [78]:
# user credentials

user='name@offerupnow.com'
password=''
account='offerup'

In [79]:
# setting SQL file path for functions

rev = '/Users/leon.zhao/Desktop/SQL/MBR/mbr_get_revenue.sql'
mau = '/Users/leon.zhao/Desktop/SQL/MBR/mbr_get_mau.sql'
BingExtAds = '/Users/leon.zhao/Desktop/SQL/MBR/mbr_get_BingExternalAdsData.sql'
BndExtAds = '/Users/leon.zhao/Desktop/SQL/MBR/mbr_get_BndExternalAdsData.sql'
SellerAds = '/Users/leon.zhao/Desktop/SQL/MBR/mbr_get_SellerAdsData.sql'
IAP_retention = '/Users/leon.zhao/Desktop/SQL/MBR/mbr_get_IAP_retention.sql'
IAP_retention_bkd = '/Users/leon.zhao/Desktop/SQL/MBR/mbr_get_IAP_retention_bkd.sql'
ipromo_bkd = '/Users/leon.zhao/Desktop/SQL/MBR/mbr_get_item_promo_bkd_new.sql'
ipromo_bkd_newsubs = '/Users/leon.zhao/Desktop/SQL/MBR/mbr_get_item_promo_new_sub.sql'
tnc = '/Users/leon.zhao/Desktop/SQL/MBR/mbr_get_tnc.sql'
pcb = '/Users/leon.zhao/Desktop/SQL/MBR/mbr_get_promo_clicked_buy.sql'
pro_screen = '/Users/leon.zhao/Desktop/SQL/MBR/mbr_get_promo_screen.sql'
cvt = '/Users/leon.zhao/Desktop/SQL/MBR/mbr_get_cvt_source.sql'
subs_bkd = '/Users/leon.zhao/Desktop/SQL/MBR/mbr_get_subs_bkd.sql'

In [80]:
import snowflake.connector
import pandas as pd
import numpy as np



class SnowflakeSession:

    def __init__(self, user, password, account):

        # Connect and get cursor
        print("> Connecting to Snowflake")
        self.ctx = snowflake.connector.connect(user=user,
                                                password=password,
                                                account=account, )
        self.cs = self.ctx.cursor()

        # Set up cursor
        self.cs.execute("use warehouse analytics_wh;")
        self.cs.execute("use role marketing;")
        self.cs.execute("Alter session set timezone = 'America/Los_Angeles';") # for TZ aware fields

    @staticmethod
    def parse_query_file(sql_query_file):
        """Parse SQL file and return query as string."""
        
        # Reading of the SQL statement from sql_query.txt file
#         print("> Reading SQL statement from file")
        fd = open(sql_query_file, 'r')
        sqlFile = fd.read()
        fd.close()

        # Extraction of the SQL statement (split on ';')
        # Get only the first statement.
        sql_sentence_list = sqlFile.split(';')
        sql_query = [x for x in sql_sentence_list if x][0]

        return sql_query

    def execute_query(self, qry):
#         print("> Querying database with:")
#         print(qry)
#         print("\n> Fetching data ...")
        self.cs.execute(qry)
        colnames = [x[0] for x in self.cs.description]
        rows = self.cs.fetchall()
        return rows, colnames

    @staticmethod
    def result_to_df(rows, colnames):
        print("> Saving to dataframe ...")
        df =  pd.DataFrame(rows, columns=colnames)
        df.replace({r'\r': ''}, regex=True, inplace=True)
        return df


    @staticmethod
    def save_df_to_csv(df, fname):
        print("\n> Saving to file ...")
        if '.csv' in fname:
            separator = ','
        else:
            separator = '\t'
        df.to_csv(fname, index=False, sep=separator, header=True, encoding='utf-8')

    def close_session(self):
        print("\n> Closing session ...")
        self.cs.close()
        self.ctx.close()
        
s = SnowflakeSession(user, password, account)

> Connecting to Snowflake


In [81]:
class MBR:
    @staticmethod
    def get_revenue(path):
        
        try:
            data = s.parse_query_file(path)
            rows, colnames = s.execute_query(data)
            revenue_data = pd.DataFrame(data = rows, columns = colnames)
            revenue_data = revenue_data.pivot_table(index=['PLATFORM', 'REVENUE_SOURCE'], values=['GROSS_REVENUE'], columns=['DATE_MONTH'], aggfunc=[np.sum])
        finally:
            print('\n> Revenue Done\n> ')
        return revenue_data.to_csv('all_time_revenue.csv')

    @staticmethod
    def get_mau(path):
        
        try:
            data = s.parse_query_file(path)
            rows, colnames = s.execute_query(data)
            mau_data = pd.DataFrame(data = rows, columns = colnames)
            mau_data = mau_data.pivot_table(index=['PLATFORM'], values=['NUM'], columns=['MONTH'], aggfunc=[np.sum])
        finally:
            print('\n> MAU Done\n> ')
        return mau_data.to_csv('updated_mau.csv')

    @staticmethod
    def get_BingExternalAdsData(path):
        
        try:
            data = s.parse_query_file(path)
            rows, colnames = s.execute_query(data)
            bing_data = pd.DataFrame(data = rows, columns = colnames)
            bing_data_1 = bing_data.pivot_table(index=['MONTH'], values=['AD_DENSITY', 'CPC', 'ESTIMATED_REVENUE'], columns=['PLATFORM'], aggfunc=[np.sum])
            bing_data_2 = bing_data.pivot_table(index=['MONTH'], values=['AD_CTR', 'AD_IMPRESSIONS', 'CLICKS'], columns=['PLATFORM'], aggfunc=[np.sum])
            bing_data_3 = bing_data.pivot_table(index=['MONTH'], values=['CTR', 'AD_REQUESTS'], columns=['PLATFORM'], aggfunc=[np.sum])
            bing_data_f = pd.concat([bing_data_1, bing_data_2, bing_data_3], axis=1, sort=False)
        finally:
            print('\n> Bing Data Done\n> ')
        return bing_data_f.to_csv('updated_BingExternalAdsData.csv')

    @staticmethod
    def get_BndExternalAdsData(path):
        
        try:
            data = s.parse_query_file(path)
            rows, colnames = s.execute_query(data)
            bnd_data = pd.DataFrame(data = rows, columns = colnames)
            bnd_data = bnd_data.pivot_table(index=['MONTH'], values=['ADS_REQUESTED', 'ADS_RETURNED', 'AD_REQUESTS'], columns=['PLATFORM'], aggfunc=[np.sum])
        finally:
            print('\n> Bnd Ext Done\n> ')
        return bnd_data.to_csv('updated_BndExternalAdsData.csv')

    @staticmethod
    def get_SellerAdsData(path):
        
        try:
            data = s.parse_query_file(path)
            rows, colnames = s.execute_query(data)
            sa_data = pd.DataFrame(data = rows, columns = colnames)
            sa_data = sa_data.pivot_table(index=['MONTH'], values=['ACTIVE_ITEMS', 'ACTIVE_ITEMS_FROM_PURCHASERS', 'ACTIVE_ITEMS_WITH_PURCHASE_FROM_ADVERTISER', 'ACTIVE_SELLERS', 'USERS_PURCHASING'], columns=['PLATFORM'], aggfunc=[np.sum])
        finally:
            print('\n> SellerAds Done\n> ')
        return sa_data.to_csv('updated_SellerAdsData.csv')

    @staticmethod
    def get_IAP_retention(path):
        
        try:
            data = s.parse_query_file(path)
            rows, colnames = s.execute_query(data)
            iap_data = pd.DataFrame(data = rows, columns = colnames)
            iap_data_bk = iap_data.pivot_table(index=['ACTUAL_MONTH'], values=['RETAINED_IAPER', 'ADVERTISERS_RETURNING_FROM_PRIOR_MONTH'], columns=['PLATFORM'], aggfunc=[np.sum])
            iap_data_ttl = iap_data.pivot_table(index=['ACTUAL_MONTH'], values=['RETAINED_IAPER', 'ADVERTISERS_RETURNING_FROM_PRIOR_MONTH'], aggfunc=[np.sum])
        finally:
            print('\n> IAP Retention Done\n> ')
        return iap_data_bk.to_csv('updated_iapRetentionData.csv'), iap_data_ttl.to_csv('updated_iapRetentionData_total.csv')
    
    @staticmethod
    def get_IAP_retention_bkd(path):
        
        try:
            data = s.parse_query_file(path)
            rows, colnames = s.execute_query(data)
            iap_data = pd.DataFrame(data = rows, columns = colnames)
            iap_data_bk = iap_data.pivot_table(index=['ACTUAL_MONTH'], values=['RETAINED_IAPER', 'ADVERTISERS_RETURNING_FROM_PRIOR_MONTH'], columns=['PROMO_TYPE'], aggfunc=[np.sum])            
        finally:
            print('\n> IAP Retention Done\n> ')
        return iap_data_bk.to_csv('updated_iapRetentionData_bkd.csv')
    
    
    @staticmethod
    def get_itempromo_bkd(path):
        
        try:
            data = s.parse_query_file(path)
            rows, colnames = s.execute_query(data)
            ip_data = pd.DataFrame(data = rows, columns = colnames)
            print(rows, colnames)
            ip_data_f = ip_data.pivot_table(index=['MONTH'], values=['TRANSACTIONS', 'A_LA_CARTE_ADVERTISERS', 'FIRST_TIME_ADVERTISERS', 'REVENUE'], columns=['PLATFORM'], aggfunc=[np.sum])
            ip_data_ttl = ip_data.pivot_table(index=['MONTH'], values=['TRANSACTIONS', 'A_LA_CARTE_ADVERTISERS', 'FIRST_TIME_ADVERTISERS', 'REVENUE'], aggfunc=[np.sum])
            ip_data_bkd = ip_data.pivot_table(index=['MONTH'], values=['REVENUE', 'TRANSACTIONS', 'A_LA_CARTE_ADVERTISERS', 'FIRST_TIME_ADVERTISERS'], columns=['PLATFORM', 'PROMO_TYPE'], aggfunc=[np.sum])
        finally:
            print('\n> Itempromo Done\n> ')
        return ip_data_f.to_csv('testing_itempromo.csv'), ip_data_ttl.to_csv('testing_itempromo_total.csv'), ip_data_bkd.to_csv('testing_itempromo_breakdown.csv')
#         return ip_data_f.to_csv('updated_itempromo.csv'), ip_data_ttl.to_csv('updated_itempromo_total.csv'), ip_data_bkd.to_csv('updated_itempromo_breakdown.csv')
    
    
    @staticmethod
    def get_subTnC(path):
        
        try:
            data = s.parse_query_file(path)
            rows, colnames = s.execute_query(data)
            tnc_data = pd.DataFrame(data = rows, columns = colnames)
            tnc_data = tnc_data.pivot_table(index=['DATE'], values=['SUBSCRIPTION_TNC_SCREENVIEWS', 'SUBSCRIPTION_TNC_USERS'], columns=['PLATFORM'], aggfunc=[np.sum])
        finally:
            print('\n> SubTnC Done\n> ')
#         return tnc_data.to_csv('testing_subTnC.csv')
        return tnc_data.to_csv('updated_subTnC.csv')
    
    
    @staticmethod
    def get_promo_clicked_buy(path):
        
        try:
            data = s.parse_query_file(path)
            rows, colnames = s.execute_query(data)
            pcb_data = pd.DataFrame(data = rows, columns = colnames)
            pcb_data = pcb_data.pivot_table(index=['DATE'], values=['EVENTS_CLICKED', 'USERS_CLICKED'], columns=['PLATFORM', 'PROMO_TYPE_CLICKED'], aggfunc=[np.sum])
        finally:
            print('\n> Promo Clicked Buy Done\n> ')
#         return pcb_data.to_csv('test_promo_clicked_buy.csv')
        return pcb_data.to_csv('updated_promo_clicked_buy.csv')
    
    @staticmethod
    def get_promo_screen(path):
        
        try:
            data = s.parse_query_file(path)
            rows, colnames = s.execute_query(data)
            gpc_data = pd.DataFrame(data = rows, columns = colnames)
            gpc_data = gpc_data.pivot_table(index=['DATE'], values=['ITEMPROMOSELECTION_SCREENVIEWS', 'ITEMPROMOSELECTION_USERS'], columns=['PLATFORM'], aggfunc=[np.sum])
        finally:
            print('\n> Promo Screen Done\n> ')
#         return gpc_data.to_csv('test_promo_screen.csv')
        return gpc_data.to_csv('updated_promo_screen.csv')
    
    @staticmethod
    def get_cvt_source(path):
        
        try:
            data = s.parse_query_file(path)
            rows, colnames = s.execute_query(data)
            cvt_data = pd.DataFrame(data = rows, columns = colnames)
            cvt_data = cvt_data.pivot_table(index=['DATE'], values=['ITEMPERFORMANCE_VIEWED', 'EDITITEM3_VIEWED', 'MYOFFERSSELLING_VIEWED', 'ITEMDASHBOARD_VIEWED', 'POSTITEMCONFIRMATION_VIEWED', 'EDITITEMCONFIRMATION_VIEWED'], columns=['SYSTEM_NAME'], aggfunc=[np.sum])
        finally:
            print('\n> Cvt Source Done\n> ')
#         return cvt_data.to_csv('test_cvt_source.csv')
        return cvt_data.to_csv('updated_cvt_source.csv')
    
    @staticmethod
    def get_subs_bkd(path):
        
        try:
            data = s.parse_query_file(path)
            rows, colnames = s.execute_query(data)
            subs = pd.DataFrame(data = rows, columns = colnames)
            subs = subs.pivot_table(index=['DATE'], values=['NUM_FREE_TRIALS', 'NUM_SUBSCRIBERS'], columns=['PLATFORM', 'SUB_STATUS'], aggfunc=[np.sum])
        finally:
            print('\n> Subs Breakdown Done\n>')
        return subs.to_csv('updated_Subs_bkd.csv')

# temps    
    @staticmethod
    def get_itempromo_new_subs(path):
        
        try:
            data = s.parse_query_file(path)
            rows, colnames = s.execute_query(data)
            iap_data = pd.DataFrame(data = rows, columns = colnames)
            iap_data_bk = iap_data.pivot_table(index=['MONTH'], values=['TRANSACTIONS', 'INTERNALADS_ADVERTISERS', 'FIRST_TIME_ADVERTISERS', 'REVENUE'], aggfunc=[np.sum])            
        finally:
            print('\n> IAP Retention Done\n> ')
        return iap_data_bk.to_csv('updated_itempromo_newsubs.csv')
    
    
    
    
m = MBR()
    

In [114]:
def get_mbr():
    try:
        m.get_revenue(rev)
        m.get_mau(mau)
        m.get_BingExternalAdsData(BingExtAds)
        m.get_BndExternalAdsData(BndExtAds)
        m.get_SellerAdsData(SellerAds)
        m.get_IAP_retention(IAP_retention)
        m.get_IAP_retention_bkd(IAP_retention_bkd)
#         m.get_itempromo_bkd(ipromo_bkd)
        m.get_subTnC(tnc)
        m.get_promo_clicked_buy(pcb)
        m.get_promo_screen(pro_screen)
        m.get_cvt_source(cvt)
    except:
        print('THEY SEE ME ROLLING....AND NOT ANY MORE')

In [76]:
m.get_itempromo_new_subs(ipromo_bkd_newsubs)


> IAP Retention Done
> 


In [82]:
m.get_cvt_source(cvt)


> Cvt Source Done
> 


In [115]:
get_mbr()


> Revenue Done
> 

> MAU Done
> 

> Bing Data Done
> 

> Bnd Ext Done
> 

> SellerAds Done
> 

> IAP Retention Done
> 

> Itempromo Done
> 

> SubTnC Done
> 

> Promo Clicked Buy Done
> 

> Promo Screen Done
> 

> Cvt Source Done
> 


# Archives